In [17]:
import os
import pandas as pd
import datetime
from datetime import date
import time
import numpy as np

## Download [this query on Quarry](https://quarry.wmcloud.org/query/59058) of 50k random users (with userpages who registered before 2020-10)

In [18]:
!wget https://quarry.wmcloud.org/run/585296/output/0/tsv -O user_reg_with_userpages.tsv

--2021-10-11 04:12:27--  https://quarry.wmcloud.org/run/585296/output/0/tsv
Resolving quarry.wmcloud.org (quarry.wmcloud.org)... 172.16.0.164
Connecting to quarry.wmcloud.org (quarry.wmcloud.org)|172.16.0.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/tab-separated-values]
Saving to: ‘user_reg_with_userpages.tsv’

user_reg_with_userp     [  <=>               ]   1.71M  4.47MB/s    in 0.4s    

2021-10-11 04:12:28 (4.47 MB/s) - ‘user_reg_with_userpages.tsv’ saved [1795476]



In [19]:
!wc -l user_reg_with_userpages.tsv

50001 user_reg_with_userpages.tsv


In [20]:
!head user_reg_with_userpages.tsv

In [21]:
editors_df = pd.read_csv('user_reg_with_userpages.tsv', delimiter='\t', infer_datetime_format=True)
editors_df

,user_name,user_id,user_registration
0,KingRiach,27773404,20160307220704
1,Ctambo,6492102,20080221002533
2,Awisely,13820329,20110118230413
3,Shankhamita,16009368,20120101171414
4,Abhinayaprithivi,28005397,20160404204752
...,...,...,...
49995,Bandgeek1732,16336675,20120221035543
49996,Werqty12376,18321203,20130125002347
49997,Chrisboote,482416,20051007102059
49998,Mariibog,24622706,20150402172429


### Function for calculating future dates

In [22]:
def add_days(d, days):

    return d + datetime.timedelta(days=days)


In [23]:
add_days(datetime.datetime(2010,1,1),365)

datetime.datetime(2011, 1, 1, 0, 0)

In [24]:
add_days(datetime.datetime(2010,1,1),364)

datetime.datetime(2010, 12, 31, 0, 0)

## Building one SQL query for each user's edits

In [25]:
query_list = []
reg_year_mo_list = []
for index, row in editors_df.iterrows():
    user_registration = row['user_registration']
    reg_dt = datetime.datetime.strptime(str(user_registration), "%Y%m%d%H%M%S")

    start_date = add_days(reg_dt, 182)
    end_date = add_days(reg_dt, 365)
    reg_year_mo = reg_dt.strftime("%Y-%m")
    
    reg_year_mo_list.append(reg_year_mo)
    

#    query = "select " + str(row['user_id']) + " as user_id, " + str(row['user_registration']) + " as user_registration,"
#    query += "'" + str(reg_year_mo) + "' as year_mo, "
#    query += """s.* from ( select rev_id from revision_userindex where rev_actor = """ + str(row['user_id'])
#    query += " and rev_timestamp > " + start_date.strftime("%Y%m%d%H%M%S") 
#    query += " and rev_timestamp < " + end_date.strftime("%Y%m%d%H%M%S") + ") as s"

    query = "select * from revision_userindex where rev_actor = " + str(row['user_id'])
    query += " and rev_timestamp > " + start_date.strftime("%Y%m%d%H%M%S") 
    query += " and rev_timestamp < " + end_date.strftime("%Y%m%d%H%M%S") + ";"

    query_list.append(query)

In [26]:
editors_df = pd.concat([editors_df,pd.Series(query_list,name="reg_year_mo")], axis=1)
editors_df = pd.concat([editors_df,pd.Series(query_list,name="query")], axis=1)
editors_df

,user_name,user_id,user_registration,reg_year_mo,query
0,KingRiach,27773404,20160307220704,select * from revision_userindex where rev_act...,select * from revision_userindex where rev_act...
1,Ctambo,6492102,20080221002533,select * from revision_userindex where rev_act...,select * from revision_userindex where rev_act...
2,Awisely,13820329,20110118230413,select * from revision_userindex where rev_act...,select * from revision_userindex where rev_act...
3,Shankhamita,16009368,20120101171414,select * from revision_userindex where rev_act...,select * from revision_userindex where rev_act...
4,Abhinayaprithivi,28005397,20160404204752,select * from revision_userindex where rev_act...,select * from revision_userindex where rev_act...
...,...,...,...,...,...
49995,Bandgeek1732,16336675,20120221035543,select * from revision_userindex where rev_act...,select * from revision_userindex where rev_act...
49996,Werqty12376,18321203,20130125002347,select * from revision_userindex where rev_act...,select * from revision_userindex where rev_act...
49997,Chrisboote,482416,20051007102059,select * from revision_userindex where rev_act...,select * from revision_userindex where rev_act...
49998,Mariibog,24622706,20150402172429,select * from revision_userindex where rev_act...,select * from revision_userindex where rev_act...


## Querying the database with a manually-written query

In [27]:
import pymysql

# Host urls are like {wiki}.{analytics,web}.db.svc.wikimedia.cloud
host = "enwiki.web.db.svc.wikimedia.cloud"
credentials = "~/.my.cnf"

query = """    select rev_timestamp from revision_userindex
               where rev_actor = 54
               and rev_timestamp > 20121003204752
               and rev_timestamp < 20121104204752
            
        """

#query = "show tables"

#query = "describe revision_userindex"

#query = "select * from user limit 1"


conn = pymysql.connect(
    host=host,
    read_default_file=credentials,
    database="enwiki_p"
)

with conn.cursor() as cur:
        cur.execute(query)
        data = cur.fetchall()        
conn.close()


data

((b'20121007153954',),
 (b'20121007154210',),
 (b'20121008155455',),
 (b'20121016134012',),
 (b'20121018031929',),
 (b'20121018031950',))

Processing these results (user_id, reg_date, reg_date_month, edit timestamp in bytestring)

In [28]:
print("Number of edits between dates:", len(data))

for edit in data:
    print("Edit made on: ", datetime.datetime.strptime(edit[0].decode(),"%Y%m%d%H%M%S"))

Number of edits between dates: 6
Edit made on:  2012-10-07 15:39:54
Edit made on:  2012-10-07 15:42:10
Edit made on:  2012-10-08 15:54:55
Edit made on:  2012-10-16 13:40:12
Edit made on:  2012-10-18 03:19:29
Edit made on:  2012-10-18 03:19:50


In [29]:
conn = pymysql.connect(
    host=host,
    read_default_file=credentials,
    database="enwiki_p"
)

data_list = []

for index, row in editors_df.iterrows():
    with conn.cursor() as cur:
        cur.execute(row['query'])
        data = cur.fetchall()

        result = {}
        
        result['edits'] = len(data)
        
        result['user_id'] = row['user_id']
        result['reg_date'] = row['user_registration']

        if result['edits'] == 0:
            result['survived'] = 0
        else:
            result['survived'] = 1
            
        data_list.append(result)
            
        if(len(data_list) % 500 == 0):
            raise Exception("Stopping at 500 rows")

conn.close()

Exception: Stopping at 500 rows

In [ ]:
pd.DataFrame(data_list).sort_values("edits")

### Query all 50,000 editor histories

In [ ]:
conn = pymysql.connect(
    host=host,
    read_default_file=credentials,
    database="enwiki_p"
)

data_list = []

for index, row in editors_df.iterrows():
    with conn.cursor() as cur:
        cur.execute(row['query'])
        data = cur.fetchall()

        result = {}
        
        result['edits'] = len(data)
        
        result['user_id'] = row['user_id']
        result['reg_date'] = row['user_registration']

        if result['edits'] == 0:
            result['survived'] = 0
        else:
            result['survived'] = 1
            
        data_list.append(result)
            
        if(len(data_list) % 1000 == 0):
            print(len(data_list))

conn.close()

In [ ]:
retention_df = pd.DataFrame(data_list)
len(retention_df)
retention_df.sort_values("edits")

In [ ]:
def datetime_to_year_mo(dt):
    return datetime.datetime.strftime(dt, "%Y-%m")

In [ ]:
retention_df['reg_date'] = pd.to_datetime(retention_df['reg_date'], format="%Y%m%d%H%M%S")
retention_df['reg_date_year_mo'] = retention_df['reg_date'].apply(datetime_to_year_mo)
retention_df

### Plot results

In [ ]:
retention_group = retention_df.groupby(["reg_date_year_mo"])
monthly_averages = retention_group.aggregate({"survived":np.mean})
#monthly_averages = monthly_averages.query("reg_date >= '2006-01'")
fig = monthly_averages.plot(figsize=(12,8))


## Assignment 2

This does not check to see if the editor actually made an edit to an article, only that they registered. Expand to filter out every user who did not make an edit in their first six months. You will need to either keep the first query to the revision table for every user and add a second query for every user to filter out those who did not make an edit --- or (recommended) you can modify the first query to revision table in the database to do return a result for all 12 months, which will let you measure survival and do the filtering. Remember that you must join with the page table to filter by namespaces (articles are namespace 0). Here is an example query for getting a user's edits: https://quarry.wmcloud.org/query/59069


In [ ]:
editors_df = pd.read_csv('user_reg_with_userpages.tsv', delimiter='\t', infer_datetime_format=True)
editors_df

In [ ]:
query_list = []
reg_year_mo_list = []
for index, row in editors_df.iterrows():
    user_registration = row['user_registration']
    reg_dt = datetime.datetime.strptime(str(user_registration), "%Y%m%d%H%M%S")

    start_date = add_days(reg_dt, 0)
    end_date = add_days(reg_dt, 183)
    reg_year_mo = reg_dt.strftime("%Y-%m")
    
    reg_year_mo_list.append(reg_year_mo)
    

#    query = "select " + str(row['user_id']) + " as user_id, " + str(row['user_registration']) + " as user_registration,"
#    query += "'" + str(reg_year_mo) + "' as year_mo, "
#    query += """s.* from ( select rev_id from revision_userindex where rev_actor = """ + str(row['user_id'])
#    query += " and rev_timestamp > " + start_date.strftime("%Y%m%d%H%M%S") 
#    query += " and rev_timestamp < " + end_date.strftime("%Y%m%d%H%M%S") + ") as s"

#     query = "select user_name, user_id, user_registration from page" from revision_userindex where rev_actor = " + str(row['user_id'])
#     query += " inner join revision_userindex on page_title = user_name"
#     query += " and rev_timestamp > " + start_date.strftime("%Y%m%d%H%M%S") 
#     query += " and rev_timestamp < " + end_date.strftime("%Y%m%d%H%M%S") + ";"

    query = "select * from revision_userindex where rev_actor = " + str(row['user_id'])
    query += " and rev_timestamp > " + start_date.strftime("%Y%m%d%H%M%S") 
    query += " and rev_timestamp < " + end_date.strftime("%Y%m%d%H%M%S") + ";"
    

    query_list.append(query)

In [ ]:
editors_df = pd.concat([editors_df,pd.Series(query_list,name="reg_year_mo")], axis=1)
editors_df = pd.concat([editors_df,pd.Series(query_list,name="query")], axis=1)
editors_df

In [ ]:
conn = pymysql.connect(
    host=host,
    read_default_file=credentials,
    database="enwiki_p"
)

data_list = []

for index, row in editors_df.iterrows():
    with conn.cursor() as cur:
        cur.execute(row['query'])
        data = cur.fetchall()

        result = {}
        
        result['edits'] = len(data)
        
        result['user_id'] = row['user_id']
        result['reg_date'] = row['user_registration']

        if result['edits'] == 0:
            result['survived'] = 0
        else:
            result['survived'] = 1
            
        data_list.append(result)
            
        if(len(data_list) % 1000 == 0):
            print(len(data_list))

conn.close()

In [ ]:
retention_df = pd.DataFrame(data_list)
len(retention_df)
retention_df.sort_values("edits")

In [ ]:
retention_df = retention_df[retention_df['edits'] > 0]
retention_df